<a href="https://www.kaggle.com/code/kapturovalexander/kapturov-s-solution-of-leash-bio-1?scriptVersionId=170652764" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

![](https://www.kaggle.com/competitions/67356/images/header)

### My notebook is enspired by Andrew D. Blevins and his notebook [Leash Tutorial - ECFPs and Random Forest](https://www.kaggle.com/code/andrewdblevins/leash-tutorial-ecfps-and-random-forest)

### I will change all his code in my notebook later after I completely figure out with competition

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leash-BELKA/sample_submission.csv
/kaggle/input/leash-BELKA/train.parquet
/kaggle/input/leash-BELKA/test.parquet
/kaggle/input/leash-BELKA/train.csv
/kaggle/input/leash-BELKA/test.csv


In [2]:
%%time
!pip install rdkit
!pip install duckdb
print('OK')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 60.2 MB/s eta 0:00:00
OK
CPU times: user 703 ms, sys: 122 ms, total: 825 ms
Wall time: 35.3 s


![](https://www.chitkara.edu.in/blogs/wp-content/uploads/2023/03/Medicinal-Chemistry-vs-Pharmaceutics.jpg)

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import duckdb
import optuna
from optuna.samplers import TPESampler
import pickle
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import average_precision_score

In [4]:
%%time
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 30000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2min 53s, sys: 9.03 s, total: 3min 2s
Wall time: 55 s


In [5]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,215644791,O=C(Nc1cccc([N+](=O)[O-])c1C(=O)O)OCC1c2ccccc2...,Nc1cn[nH]c1,Cc1csc(N)n1,Cc1csc(Nc2nc(Nc3cn[nH]c3)nc(Nc3cccc([N+](=O)[O...,BRD4,0
1,246970016,O=C(O)C[C@@H](Cc1ccc(Cl)cc1)NC(=O)OCC1c2ccccc2...,NCC1CCC(C(F)F)CC1,Nc1nc(Br)cn2ccnc12,O=C(C[C@@H](Cc1ccc(Cl)cc1)Nc1nc(NCC2CCC(C(F)F)...,sEH,0
2,208666735,O=C(Nc1cccc(Br)c1C(=O)O)OCC1c2ccccc2-c2ccccc21,Cl.NCC1CCOC2(CCOCC2)C1,Nc1nnc(S)s1,O=C(N[Dy])c1c(Br)cccc1Nc1nc(NCC2CCOC3(CCOCC3)C...,HSA,0
3,275331846,O=C(O)C[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1cc...,NCCc1coc2ccccc12,COc1ccc2nc(N)sc2c1,COc1ccc2nc(Nc3nc(NCCc4coc5ccccc45)nc(N[C@@H](C...,BRD4,0
4,97353741,O=C(N[C@@H](Cc1ccc(F)c(F)c1)C(=O)O)OCC1c2ccccc...,Cc1ccc(N)nn1,CC(C)(C)c1cc(N)n[nH]1,Cc1ccc(Nc2nc(Nc3cc(C(C)(C)C)[nH]n3)nc(N[C@@H](...,BRD4,0


![](https://previews.123rf.com/images/olegdudko/olegdudko1712/olegdudko171200585/91056161-chemistry-science-formula-and-tablets-medicine-symbol.jpg)

In [6]:
%%time
# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

CPU times: user 1min 44s, sys: 2.24 s, total: 1min 46s
Wall time: 1min 46s


In [7]:
%%time
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

# Create and train the random forest model
rf_model = CatBoostClassifier(iterations=500, random_state=27)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.8f}")

Learning rate set to 0.101594
0:	learn: 0.6587357	total: 115ms	remaining: 57.4s
1:	learn: 0.6294139	total: 179ms	remaining: 44.6s
2:	learn: 0.6046120	total: 232ms	remaining: 38.4s
3:	learn: 0.5868222	total: 284ms	remaining: 35.2s
4:	learn: 0.5713043	total: 340ms	remaining: 33.6s
5:	learn: 0.5588001	total: 388ms	remaining: 32s
6:	learn: 0.5439851	total: 436ms	remaining: 30.7s
7:	learn: 0.5358505	total: 492ms	remaining: 30.3s
8:	learn: 0.5254938	total: 568ms	remaining: 31s
9:	learn: 0.5184722	total: 619ms	remaining: 30.3s
10:	learn: 0.5123025	total: 667ms	remaining: 29.7s
11:	learn: 0.5056592	total: 716ms	remaining: 29.1s
12:	learn: 0.5006843	total: 761ms	remaining: 28.5s
13:	learn: 0.4967700	total: 807ms	remaining: 28s
14:	learn: 0.4932327	total: 852ms	remaining: 27.5s
15:	learn: 0.4874587	total: 904ms	remaining: 27.3s
16:	learn: 0.4831438	total: 954ms	remaining: 27.1s
17:	learn: 0.4769952	total: 1s	remaining: 26.8s
18:	learn: 0.4726366	total: 1.05s	remaining: 26.7s
19:	learn: 0.4693886

![](https://ars.els-cdn.com/content/image/1-s2.0-S2001037020304967-gr3.jpg)

In [8]:
%%time
# Process the test.parquet file chunk by chunk
test_file = '/kaggle/input/leash-BELKA/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
# Wrap the loop with tqdm to display progress
for df_test in tqdm(pd.read_csv(test_file, chunksize=10_000)):
    
    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    probabilities = rf_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))

168it [1:11:31, 25.55s/it]

CPU times: user 1h 11min 9s, sys: 41 s, total: 1h 11min 50s
Wall time: 1h 11min 31s


![](https://m.media-amazon.com/images/I/41Xf5xwc25L._SR600%2C315_PIWhiteStrip%2CBottomLeft%2C0%2C35_SCLZZZZZZZ_FMpng_BG255%2C255%2C255.jpg)